In [1]:
configs = {'n_residual_layers' : 5}

In [2]:
import mx_layers as layers

_convolution = lambda X : layers.convolution(X=X, n_filters=16, kernel_shape=(5, 5), stride=(1, 1), pad=(4, 4))
# _convolution = lambda X : layers.convolution(X=X, n_filters=16, kernel_shape=(9, 9), stride=(1, 1), pad=(4, 4))
network = layers.variable('data')
for index in range(2):
    network = _convolution(network)
    network = layers.ReLU(network)
    # TODO impact of pooling
    network = layers.pooling(X=network, mode='maximum', kernel_shape=(2, 2), stride=(2, 2), pad=(0, 0))

shared_weight = layers.variable('shared_weight')
shared_gamma = layers.variable('shared_gamma')
shared_beta = layers.variable('shared_beta')

_convolution = lambda X : layers.convolution(
    X=X, n_filters=16, kernel_shape=(3, 3), stride=(1, 1), pad=(2, 2), weight=shared_weight, no_bias=True
#     X=X, n_filters=16, kernel_shape=(5, 5), stride=(1, 1), pad=(2, 2), weight=shared_weight, no_bias=True
)
for index in range(configs['n_residual_layers']):
    network = layers.batch_normalization(network, beta=shared_beta, gamma=shared_gamma, fix_gamma=False)
    network = layers.ReLU(network)
    network = _convolution(network)

network = layers.pooling(X=network, mode='average', kernel_shape=(14, 14), stride=(1, 1), pad=(0, 0))
network = layers.flatten(network)
network = layers.fully_connected(X=network, n_hidden_units=10)

In [3]:
from data_utilities import load_mnist
stretched = load_mnist('stretched_mnist/', scale=1, shape=(1, 56, 56))

In [4]:
import cPickle as pickle
parameters, states = pickle.load(open('parameters/residual-network-on-standard-mnist-6', 'rb'))

In [5]:
from scipy.misc import imresize
import mxnet as mx
context = mx.cpu(0)
n_samples = 1000
arg_shapes, _, aux_state_shapes = network.infer_shape(data=(n_samples, 1, 56, 56))
args = network.list_arguments()
args = {arg : mx.nd.zeros(shape) for arg, shape in zip(args, arg_shapes)}
for key, value in args.items():
    print key, value.shape
    if ('convolution' in key and 'weight' in key) or key == 'shared_weight':
        weight = parameters[key]
        IN, OUT, L, L = weight.shape
        for i in range(IN):
            for j in range(OUT):
                value[i][j][:] = imresize(weight[i][j], (L * 2 - 1, L * 2 - 1))
    elif key != 'data':
        value[:] = mx.nd.array(parameters[key], context)
args['data'] = mx.nd.array(stretched[4][:n_samples], context)
aux_states = {key : mx.nd.array(value, context) for key, value in states.items()}

shared_beta (16L,)
shared_gamma (16L,)
convolution0_bias (16L,)
fullyconnected0_weight (10L, 3136L)


MXNetError: [23:55:55] src/ndarray/ndarray.cc:239: Check failed: from.shape() == to->shape() operands shape mismatchfrom.shape = (10,16) to.shape=(10,3136)

Stack trace returned 10 entries:
[bt] (0) /home/alex/python_official/mxnet/python/mxnet/../../lib/libmxnet.so(_ZN4dmlc15LogMessageFatalD1Ev+0x3c) [0x7fc04890ed4c]
[bt] (1) /home/alex/python_official/mxnet/python/mxnet/../../lib/libmxnet.so(_ZN5mxnet10CopyFromToERKNS_7NDArrayEPS0_i+0x105) [0x7fc049297605]
[bt] (2) /home/alex/python_official/mxnet/python/mxnet/../../lib/libmxnet.so(+0xf7f094) [0x7fc0492e8094]
[bt] (3) /home/alex/python_official/mxnet/python/mxnet/../../lib/libmxnet.so(MXImperativeInvoke+0x2cd) [0x7fc04918a9dd]
[bt] (4) /home/alex/python_official/anaconda2/lib/python2.7/lib-dynload/_ctypes.so(ffi_call_unix64+0x4c) [0x7fc077f5431c]
[bt] (5) /home/alex/python_official/anaconda2/lib/python2.7/lib-dynload/_ctypes.so(ffi_call+0x1f5) [0x7fc077f53a75]
[bt] (6) /home/alex/python_official/anaconda2/lib/python2.7/lib-dynload/_c

In [ ]:
executor = network.bind(context, args, aux_states=aux_states)

In [ ]:
import numpy as np
scores = executor.forward(is_train=False)[0].asnumpy()
predictions = np.argmax(scores, axis=1)
sum(predictions - stretched[5][:n_samples] != 0)